Q. 타이타닉 생존자 예측 데이터 세트 train.csv에 대하여 다음 사항을 수행하세요.
- 일괄 전처리 사용자 함수 transform_features(df) 작성
- dt, lr, rf 모델링 및 평가(정확도)
- dt_clf , folds=5 적용하여 KFold 교차검증 수행
- dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행
- GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
  - parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
  - dt_clf, scoring='accuracy', cv=5 적용

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

drive.mount('/content/drive')